<a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"><img src = "https://i.pinimg.com/474x/70/3a/e9/703ae962d5e73b8c43cdfefe8568e6a4--maple-leaf-tattoos-canada-maple-leaf-tattoo.jpg" width = 200> </a> 

# Question 1

##### I scraped the provided Wikipedia page, in order to obtain the data that is in the table of postal codes and transformed it into a pandas dataframe

In [50]:
import pandas as pd
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url, header=0)[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


##### We have to ignore the Boroughs that are **Not assigned** from the dataframe.

In [66]:
mask = df['Borough'].isin(['Not assigned'])
cn_zip = df[~mask]
cn_zip.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [51]:
df.shape

(180, 3)

# Question 2
### Add location data from csv

In [62]:
LL= pd.read_csv('http://cocl.us/Geospatial_data')
LL.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### Merged both columns and removed duplicated Postal Code column from the csv

In [90]:
new = cn_zip.merge(LL,left_on='Postal code', right_on='Postal Code')
df2 = new.drop(columns = 'Postal Code')
df2.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


# Question 3
### Clustering

In [74]:
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries were imported!')

Solving environment: done

# All requested packages already installed.

Libraries were imported!


In [91]:
k=3
can_clusters = df2.drop(['Postal code','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(can_clusters)
kmeans.labels_
df2.insert(0, 'Cluster Labels', kmeans.labels_)

In [92]:
df2.head()

,Cluster Labels,Postal code,Borough,Neighborhood,Latitude,Longitude
0,1,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,2,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,0,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


##### Creating clusters and mapping them

In [93]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude

toronto_map = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10.5)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, neighborhood, cluster in zip(df2['Latitude'], df2['Longitude'], df2['Neighborhood'], df2['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.3).add_to(toronto_map)
       
toronto_map